In [1]:
import argparse
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from dataclasses import dataclass
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from model import GModel
from myutils import roc_auc, translate_result, filter_target
from load_data import load_data
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
@dataclass
class Args:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    data: str = "ctrp"
    n_jobs: int = 5 # ← ここをコア数に応じて調整
    lr: float = 5e-4
    epochs: int = 1000

In [4]:
args = Args()
target_option = "cell"  # "cell" か "drug"

In [5]:
# データ読み込み
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

response matrix (res) shape: (470, 807)
exprs shape: (1089, 19851)
mut shape: (1089, 1667)
cna shape: (1090, 23316)
807
exprs shape: (807, 19851)
mut shape: (807, 1667)
cna shape: (807, 23316)
drug_feature shape: (470, 920)
response matrix (res) shape: (807, 470)
null_mask shape: (807, 470)


In [6]:
def run_single_model(exprs, cna, mut, drug_feature, res_mat, null_mask, target_dim, target_index, args, seed):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    val_labels = sampler.test_data[sampler.test_mask]

    model = GModel(
        adj_mat=sampler.train_data.float(),
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=args.device,
    )
    opt = Optimizer(
        model=model,
        train_data=sampler.train_data,
        test_data=sampler.test_data,
        test_mask=sampler.test_mask,
        train_mask=sampler.train_mask,
        evaluate_fun=roc_auc,
        lr=args.lr,
        epochs=args.epochs,
        device=args.device,
    ).to(args.device)
    _, true_data, predict_data = opt()
    return true_data, predict_data

In [7]:
# ターゲット次元と統計
target_dim = 0 if target_option == "cell" else 1
samples = res.shape[target_dim]
cell_sum = np.sum(res.values, axis=1)
drug_sum = np.sum(res.values, axis=0)

In [8]:
# 並列実行用ラッパー
def process_target(seed, target_index):
    try:
        return run_single_model(
            exprs=exprs,
            cna=cna,
            mut=mut,
            drug_feature=drug_feature,
            res_mat=res.values,
            null_mask=null_mask,
            target_dim=target_dim,
            target_index=target_index,
            args=args,
            seed=seed,
        )
    except Exception as e:
        print(f"❌ Failed at target {target_index}: {e}")
        return None

In [9]:
# 結果格納用
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
skipped_targets = []
passed_targets = []

# スキップチェック
for target_index in range(samples):
    label_vec = res.iloc[target_index] if target_dim == 0 else res.iloc[:, target_index]
    passed, reason, pos, neg, total = filter_target(label_vec)

    if passed:
        passed_targets.append(target_index)
    else:
        skipped_targets.append((target_index, reason, pos, neg, total))

# スキップ情報表示
print(f"\n🚫 Skipped Targets: {len(skipped_targets)}")
for idx, reason, pos, neg, total in skipped_targets:
    print(f"Target {idx}: skipped because {reason} (total={total}, pos={pos}, neg={neg})")

# 並列実行（max_njobs を args.n_jobs に変更可能）
results = Parallel(n_jobs=args.n_jobs)(
    delayed(process_target)(seed, target_index)
    for seed, target_index in enumerate(tqdm(passed_targets, desc=f"MOFGCN ({args.data} - {target_option})"))
)

# 結果の統合（None を除外）
for r in results:
    if r is not None:
        true_data, pred_data = r
        true_data_s = pd.concat([true_data_s, translate_result(true_data)], ignore_index=True)
        predict_data_s = pd.concat([predict_data_s, translate_result(pred_data)], ignore_index=True)


🚫 Skipped Targets: 42
Target 8: skipped because low_negative_ratio (total=165, pos=163, neg=2)
Target 22: skipped because low_negative_ratio (total=118, pos=116, neg=2)
Target 29: skipped because low_negative_ratio (total=87, pos=86, neg=1)
Target 30: skipped because few_total_samples (total=9, pos=9, neg=0)
Target 39: skipped because low_negative_ratio (total=14, pos=14, neg=0)
Target 84: skipped because low_negative_ratio (total=10, pos=10, neg=0)
Target 117: skipped because low_negative_ratio (total=83, pos=82, neg=1)
Target 128: skipped because low_negative_ratio (total=64, pos=64, neg=0)
Target 130: skipped because low_negative_ratio (total=98, pos=97, neg=1)
Target 142: skipped because low_positive_ratio (total=92, pos=1, neg=91)
Target 146: skipped because low_negative_ratio (total=105, pos=103, neg=2)
Target 148: skipped because low_negative_ratio (total=235, pos=234, neg=1)
Target 162: skipped because low_negative_ratio (total=189, pos=187, neg=2)
Target 228: skipped because 

MOFGCN (ctrp - cell):   0%|          | 0/765 [00:00<?, ?it/s]/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  gene = torch.from_numpy(gene).to(device)
/users/4/inoue019/DRBenchmark/MOFGCN/model.py:47: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /py

epoch:   0 loss:0.754402 auc:0.5111
epoch:  20 loss:0.433713 auc:0.5111
epoch:  40 loss:0.379290 auc:0.5333
epoch:  60 loss:0.363335 auc:0.6222
epoch:  80 loss:0.350082 auc:0.6667
epoch: 100 loss:0.329894 auc:0.7333
epoch: 120 loss:0.304167 auc:0.7444
epoch: 140 loss:0.271966 auc:0.7889
epoch: 160 loss:0.237170 auc:0.7889
epoch: 180 loss:0.214871 auc:0.7667
epoch: 200 loss:0.197275 auc:0.7444
epoch: 220 loss:0.185937 auc:0.7222
epoch: 240 loss:0.179590 auc:0.7111
epoch: 260 loss:0.172608 auc:0.7222
Fit finished.
epoch:   0 loss:0.795411 auc:0.2355
epoch:  20 loss:0.416016 auc:0.8080
epoch:  40 loss:0.370948 auc:0.8370
epoch:  60 loss:0.350208 auc:0.8623
epoch:  80 loss:0.325818 auc:0.8696
epoch: 100 loss:0.298407 auc:0.8478
epoch: 120 loss:0.258172 auc:0.7283
epoch: 140 loss:0.223633 auc:0.7464
epoch: 160 loss:0.201925 auc:0.7681
epoch: 180 loss:0.188355 auc:0.7645
Fit finished.
epoch:   0 loss:0.776567 auc:0.5957
epoch:  20 loss:0.402440 auc:0.5532
epoch:  40 loss:0.372512 auc:0.6383


epoch: 140 epoch:   0 loss:0.782238 auc:0.2738
epoch:  20 loss:0.406214 auc:0.7077
epoch:  40 loss:0.373603 auc:0.7292
epoch:  60 loss:0.355650 auc:0.7077
epoch:  80 loss:0.337820 auc:0.7354
epoch: 100 loss:0.313575 auc:0.6769
epoch: 120 loss:0.284266 auc:0.5846
epoch: 140 loss:0.245266 auc:0.5754
epoch: 160 loss:0.214798 auc:0.5846
epoch: 180 loss:0.196892 auc:0.5508
Fit finished.
epoch:   0 loss:0.786769 auc:0.4429
epoch:  20 loss:0.438967 auc:0.6048
epoch:  40 loss:0.379148 auc:0.6095
epoch:  60 loss:0.362586 auc:0.5952
epoch:  80 loss:0.347537 auc:0.5810
epoch: 100 loss:0.325483 auc:0.5381
epoch: 120 loss:0.299536 auc:0.5381
epoch: 140 loss:0.263529 auc:0.5429
epoch: 160 loss:0.228709 auc:0.4905
epoch: 180 loss:0.204986 auc:0.4857
Fit finished.
epoch:   0 loss:0.782237 auc:0.3704
epoch:  20 loss:0.400696 auc:0.6032
epoch:  40 loss:0.364795 auc:0.6243
epoch:  60 loss:0.347378 auc:0.6508
epoch:  80 loss:0.319935 auc:0.6614
epoch: 100 loss:0.281100 auc:0.6138
epoch: 120 loss:0.239253 

epoch:   0 loss:0.771472 auc:0.4524
epoch:  20 loss:0.454314 auc:0.6450
epoch:  40 loss:0.380484 auc:0.6169
epoch:  60 loss:0.364574 auc:0.5671
epoch:  80 loss:0.353145 auc:0.5476
epoch: 100 loss:0.338459 auc:0.5087
epoch: 120 loss:0.315943 auc:0.4935
epoch: 140 loss:0.288432 auc:0.5065
epoch: 160 loss:0.249507 auc:0.5043
epoch: 180 loss:0.219103 auc:0.4978
Fit finished.
epoch:   0 loss:0.786006 auc:0.5844
epoch:  20 loss:0.395003 auc:0.9221
epoch:  40 loss:0.364025 auc:0.9221
epoch:  60 loss:0.344467 auc:0.9481
epoch:  80 loss:0.313894 auc:0.8745
epoch: 100 loss:0.273788 auc:0.8658
epoch: 120 loss:0.231703 auc:0.8874
epoch: 140 loss:0.205617 auc:0.8745
epoch: 160 loss:0.187896 auc:0.8528
epoch: 180 loss:0.177161 auc:0.8528
Fit finished.
epoch:   0 loss:0.754988 auc:0.5658
epoch:  20 loss:0.398674 auc:0.8388
epoch:  40 loss:0.375197 auc:0.8783
epoch:  60 loss:0.357769 auc:0.8454
epoch:  80 loss:0.341009 auc:0.8289
epoch: 100 loss:0.316050 auc:0.8322
epoch: 120 loss:0.284980 auc:0.7237


...

auc:0.4375
epoch:  40 loss:0.379420 auc:0.4167
epoch:  60 loss:0.363534 auc:0.4618
epoch:  80 loss:0.351417 auc:0.5174
epoch: 100 loss:0.333308 auc:0.6181
epoch: 120 loss:0.311428 auc:0.7049
epoch: 140 loss:0.284659 auc:0.6944
epoch: 160 loss:0.250814 auc:0.6875
epoch: 180 loss:0.219841 auc:0.6944
epoch: 200 loss:0.203339 auc:0.6979
epoch: 220 loss:0.188961 auc:0.7222
epoch: 240 loss:0.180036 auc:0.7257
epoch: 260 loss:0.173452 auc:0.7396
epoch: 280 loss:0.169313 auc:0.7361
epoch: 300 loss:0.165183 auc:0.7431
epoch: 320 loss:0.161414 auc:0.7361
epoch: 340 loss:0.159182 auc:0.7431
epoch: 360 loss:0.158168 auc:0.7465
epoch: 380 loss:0.155263 auc:0.7465
epoch: 400 loss:0.153938 auc:0.7535
epoch: 420 loss:0.152382 auc:0.7535
epoch: 440 loss:0.150449 auc:0.7361
epoch: 460 loss:0.150033 auc:0.7361
Fit finished.
epoch:   0 loss:0.799221 auc:0.6122
epoch:  20 loss:0.433373 auc:0.9541
epoch:  40 loss:0.380103 auc:0.9184
epoch:  60 loss:0.363431 auc:0.9235
epoch:  80 loss:0.349677 auc:0.9133
epo

MOFGCN (ctrp - cell):  99%|█████████▉| 760/765 [04:51<00:02,  2.44it/s]

epoch: 120 loss:0.299353 auc:0.3711
epoch: 140 loss:0.267617 auc:0.3522
epoch: 160 loss:0.236149 auc:0.3145
Fit finished.
epoch:   0 loss:0.790641 auc:0.5210
epoch:  20 loss:0.395415 auc:0.4118
epoch:  40 loss:0.362531 auc:0.4874
epoch:  60 loss:0.342472 auc:0.4888
epoch:  80 loss:0.311998 auc:0.5588
epoch: 100 loss:0.272948 auc:0.5812
epoch: 120 loss:0.230441 auc:0.5854
epoch: 140 loss:0.202292 auc:0.5602
epoch: 160 loss:0.185829 auc:0.5434
epoch: 180 loss:0.175492 auc:0.5448
epoch: 200 loss:0.169541 auc:0.5546
epoch: 220 loss:0.163937 auc:0.5504
epoch: 240 loss:0.159981 auc:0.5686
epoch: 260 loss:0.158793 auc:0.5840
epoch: 280 loss:0.155187 auc:0.5784
Fit finished.
epoch:   0 loss:0.846707 auc:0.6190
epoch:  20 loss:0.501002 auc:0.8214
epoch:  40 loss:0.391072 auc:0.8254
epoch:  60 loss:0.370007 auc:0.7222
epoch:  80 loss:0.358240 auc:0.7262
epoch: 100 loss:0.345711 auc:0.6786
epoch: 120 loss:0.327175 auc:0.5437
epoch: 140 loss:0.304915 auc:0.4762
epoch: 160 loss:0.275231 auc:0.4722


MOFGCN (ctrp - cell): 100%|██████████| 765/765 [04:53<00:00,  2.61it/s]


loss:0.352172 auc:0.4640
epoch: 100 loss:0.337773 auc:0.4602
epoch: 120 loss:0.313075 auc:0.4242
epoch: 140 loss:0.282714 auc:0.3504
epoch: 160 loss:0.248922 auc:0.3314
Fit finished.
epoch:   0 loss:0.793740 auc:0.2604
epoch:  20 loss:0.407830 auc:0.6042
epoch:  40 loss:0.374517 auc:0.6823
epoch:  60 loss:0.360743 auc:0.7188
epoch:  80 loss:0.348685 auc:0.7135
epoch: 100 loss:0.331645 auc:0.6250
epoch: 120 loss:0.306625 auc:0.5833
epoch: 140 loss:0.278812 auc:0.5677
epoch: 160 loss:0.241092 auc:0.5729
epoch: 180 loss:0.215302 auc:0.5990
Fit finished.
epoch:   0 loss:0.767088 auc:0.4951
epoch:  20 loss:0.426757 auc:0.4520
epoch:  40 loss:0.380150 auc:0.4200
epoch:  60 loss:0.365014 auc:0.4089
epoch:  80 loss:0.353760 auc:0.3608
epoch: 100 loss:0.339488 auc:0.2648
epoch: 120 loss:0.318967 auc:0.1835
epoch: 140 loss:0.291835 auc:0.1047
epoch: 160 loss:0.252805 auc:0.1059
Fit finished.
epoch:   0 loss:0.782079 auc:0.4119
epoch:  20 loss:0.417077 auc:0.5153
epoch:  40 loss:0.379001 auc:0.47

In [10]:
# Save
true_path = f"mofgcn_true_{args.data}_{target_option}.csv"
pred_path = f"mofgcn_pred_{args.data}_{target_option}.csv"
true_data_s.to_csv(true_path, index=False)
predict_data_s.to_csv(pred_path, index=False)

print(f"\n✅ Done. Results saved to:\n  - {true_path}\n  - {pred_path}")


✅ Done. Results saved to:
  - mofgcn_true_ctrp_cell.csv
  - mofgcn_pred_ctrp_cell.csv
